In [1]:
import graphblas as gb

gb.init("suitesparse", blocking=False)

from graphblas import Matrix, dtypes, unary, binary, monoid, semiring, Recorder
from graphblas.io import mmread
import numpy as np

In [117]:
M = mmread('./Графы_ИТМО/karate.mtx')
M

"M_352893"  nvals  nrows  ncols  dtype     format
gb.Matrix     156     34     34   FP64  csr (iso)
-------------------------------------------------
     0    1    2    3    4    5    6    7    8    9   ...   24   25   26   27  \
0        1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0       ...                       
1   1.0       1.0  1.0                 1.0            ...                       
2   1.0  1.0       1.0                 1.0  1.0  1.0  ...                 1.0   
3   1.0  1.0  1.0                      1.0            ...                       
4   1.0                           1.0                 ...                       
5   1.0                           1.0                 ...                       
6   1.0                 1.0  1.0                      ...                       
7   1.0  1.0  1.0  1.0                                ...                       
8   1.0       1.0                                     ...                       
9             1.0                                     ...                       
10  1.0                 1.0  1.0                      ...                       
11  1.0                                               ...                       
12  1.0            1.0                                ...                       
13  1.0  1.0  1.0  1.0                                ...                       
14                                                    ...                       
15                                                    ...                       
16                           1.0  1.0                 ...                       
17  1.0  1.0                                          ...                       
18                                                    ...                       
19  1.0  1.0                                          ...                       
20                                                    ...                       
21  1.0  1.0                                          ...                       
22                                                    ...                       
23                                                    ...       1.0       1.0   
24                                                    ...       1.0       1.0   
25                                                    ...  1.0                  
26                                                    ...                       
27            1.0                                     ...  1.0                  
28            1.0                                     ...                       
29                                                    ...            1.0        
30       1.0                                1.0       ...                       
31  1.0                                               ...  1.0  1.0             
32            1.0                           1.0       ...                       
33                                          1.0  1.0  ...            1.0  1.0   

     28   29   30   31   32   33  
0                  1.0            
1             1.0                 
2   1.0                 1.0       
3                                 
4                                 
5                                 
6                                 
7                                 
8             1.0       1.0  1.0  
9                            1.0  
10                                
11                                
12                                
13                           1.0  
14                      1.0  1.0  
15                      1.0  1.0  
16                                
17                                
18                      1.0  1.0  
19                           1.0  
20                      1.0  1.0  
21                                
22                      1.0  1.0  
23       1.0            1.0  1.0  
24                 1.0            
25                 1.0            
26       1.0                 1.0  
27                           1.0  

In [62]:
type(M)

graphblas.core.matrix.Matrix

# №1. Используя python-graphblas реализовать **наивный алгоритм**, для матрицы смежности вычисляющий и возвращающий количество треугольников неориентированного графа. <br> Функция принимает представление неориентированного графа, удобное для неё (загрузка, конвертация и проверка неориентированности реализованы отдельно).<br>  Функция возвращает число --- количество треугольников в графе. 

In [101]:
def naiveA3(A: Matrix) -> float:
    # Вычисляем A^3
    A3 = A.mxm(A.mxm(A))
    
    # Выделяем диагональ матрицы, на которой выписаны все пути длины 3
    diag = A3.diag()
          
    # Находим След и делим на 6 т.к. каждый треугольник был пересчитан 6 раз (по 2м направлениям и 3м вершинам)
    res = float(diag.reduce(gb.agg.sum).value or 0) / 6
    return res

# №2. Используя python-graphblas реализовать **наивный алгоритм с маской**, для матрицы смежности вычисляющий и возвращающий количество треугольников неориентированного графа. <br> Функция принимает представление неориентированного графа, удобное для неё (загрузка, конвертация и проверка неориентированности реализованы отдельно). <br> Функция возвращает число --- количество треугольников в графе.

In [102]:
def naiveMaskA2(A: Matrix) -> float:
    # Вычисляем A^2:
    A2 = A.mxm(A)
    
    # Вычисляем маску:
    mask = A.dup(dtype=dtypes.BOOL)

    # Применим маску к матрице A^2:
    masked_A2 = A2.dup(mask=mask)
    
    # Просуммируем все элементы матрицы, посчитав тем самым количество всех треугольников и также делим на 6
    res = float(masked_A2.reduce_scalar(gb.monoid.plus).value or 0) / 6
    return res

# №3. Используя python-graphblas реализовать **Сohen's algorithm**, вычисляющий количество треугольников неориентированного графа. <br> Функция принимает представление неориентированного графа, удобное для неё (загрузка, конвертация и проверка неориентированности реализованы отдельно).<br> Функция возвращает число --- количество треугольников в графе.

In [103]:
def cohen(A: Matrix) -> float:
    # Вычисляем нижнетреугольную матрицу A:
    L = gb.select.tril(A)
    
    # Вычисляем верхнетреугольную матрицу A:
    U = gb.select.triu(A)
    
    # Вычисляем маску:
    mask = A.dup(dtype=dtypes.BOOL)
    
    # Применим маску к произведению L * U
    masked_LU = L.mxm(U).dup(mask=mask)
    
    # Просуммируем все элементы матрицы, посчитав тем самым количество всех треугольников 
    # и также делим на 2, т.к. теперь мы учитываем вершины, но еще не учитываем направление
    res = float(masked_LU.reduce_scalar(gb.monoid.plus).value or 0) / 2
    return res

# №4. Используя python-graphblas реализовать **Sandia algorithm**, вычисляющий количество треугольников неориентированного графа. <br> Функция принимает представление неориентированного графа, удобное для неё (загрузка, конвертация и проверка неориентированности реализованы отдельно). <br> Функция возвращает число --- количество треугольников в графе.

In [104]:
def sandia(A: Matrix) -> float:
    # Вычисляем нижнетреугольную матрицу A:
    L = gb.select.tril(A)

    # Вычисляем маску L:
    mask = L.dup(dtype=dtypes.BOOL)
    
    # Применяем маску к произведению L * L
    masked_LL = L.mxm(L).dup(mask=mask)
    
    # Просуммируем все элементы матрицы и получим итоговое число треугольников без повторов
    res = float(masked_LL.reduce_scalar(gb.monoid.plus).value or 0)
    return res

In [105]:
def test(A: Matrix):
    result = [naiveA3(A), naiveMaskA2(A), cohen(A), sandia(A)]
    for ind, algo_name in enumerate(['naiveA3:', 'naiveMaskA2:', 'cohen:', 'sandia:']):
        print(algo_name, result[ind], end='; ')
    if len(np.unique(result)) != 1:
       print("All return values are not equal")
    else:
        print('naiveA3:')
        %timeit naiveA3(A)
        
        print('naiveMaskA2:')
        %timeit naiveMaskA2(A)
        
        print('cohen:')
        %timeit cohen(A)
        
        print('sandia:')
        %timeit sandia(A)

In [106]:
test(M)

naiveA3: 18093.0; naiveMaskA2: 18093.0; cohen: 18093.0; sandia: 18093.0; naiveA3:
57.4 ms ± 210 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
naiveMaskA2:
1.91 ms ± 111 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
cohen:
1.03 ms ± 28.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
sandia:
661 µs ± 5.19 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


# №5. (+2 балла) Используя python-graphblas реализовать функцию, вычисляющую для каждой вершины неориентированного графа количество треугольников, в которых она участвует. <br> Функция принимает представление неориентированного графа, удобное для неё (загрузка, конвертация и проверка неориентированности реализованы отдельно). <br> Функция возвращает массив, где для каждой вершины указано, в скольки треугольниках она участвует.

In [146]:
# Чтобы посчитать количество треугольников, в которых участвует каждая вершина, 
# нам необходимо применить наивный алгоритм и поделить для каждой вершины число на 2, 
# чтобы не считать разные направления. Я воспользуюсь алгоритмом naiveMaskA2
def triangles_for_vertices(A: Matrix) -> np.ndarray:
    # Вычисляем A^2:
    A2 = A.mxm(A)
    
    # Вычисляем маску:
    mask = A.dup(dtype=dtypes.BOOL)

    # Применим маску к матрице A^2:
    masked_A2 = A2.dup(mask=mask)
    
    # Вычисляем сумму в каждой строке и делим на 2
    res = masked_A2.reduce_columnwise(monoid.plus) / 2
    return res.to_dense(fill_value=0) 

In [147]:
triangles_for_vertices(M)

array([18., 12., 11., 10.,  2.,  3.,  3.,  6.,  5.,  0.,  2.,  0.,  1.,
        6.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  4.,  1.,  1.,
        1.,  1.,  1.,  4.,  3.,  3., 13., 15.])

# №6. (+1 балл) Скачать 10 графов в формате Matrix Market с сайта **SuiteSparse Matrix Collection** и оценить время работы всех полученных реализаций. Сделать выводы.

In [179]:
import os
for file in os.listdir('./Графы_ИТМО/матрицы/'):
    if file == '.DS_Store':
        continue
    matrix = mmread(f'./Графы_ИТМО/матрицы/{file}')
    print(f'name: {file}; nvals: {matrix.nvals}, nrows: {matrix.nrows}, ncols: {matrix.ncols}')
    test(matrix)
    print()

name: Erdos992.mtx; nvals: 15030, nrows: 6100, ncols: 6100
naiveA3: 1610.0; naiveMaskA2: 1610.0; cohen: 1610.0; sandia: 1610.0; naiveA3:
7.07 ms ± 438 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
naiveMaskA2:
591 µs ± 13 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
cohen:
695 µs ± 13.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
sandia:
648 µs ± 5.09 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

name: dictionary28.mtx; nvals: 178076, nrows: 52652, ncols: 52652
naiveA3: 74093.0; naiveMaskA2: 74093.0; cohen: 74093.0; sandia: 74093.0; naiveA3:
65.2 ms ± 8.58 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
naiveMaskA2:
4.44 ms ± 129 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
cohen:
3.48 ms ± 19.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
sandia:
2.91 ms ± 106 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

name: Erdos982.mtx; nvals: 14750, nrows: 5822, ncols: 5822
naiveA3: 1595.0; naiveMas

# Выводы: <br>
1. Даже на не очень большом количестве данных наивный алгоритм проигрывает всем остальным по скорости <br>
2. При большом количестве данных sandia работает быстрее всех <br>
3. На не очень большом количестве данных (nvals ~ 15000) алгоритм с маской показывает чуть лучшие результаты, чем sandia <br>
4. cohen за бортом, на больших данных лучше использовать sandia, на малых алгоритм с маской

# №7. (+2 балла) Реализовать генератор случайных неориентированных графов, в котором можно задавать количество вершин и степень разреженности графа. Путём генерации случайных графов различного размера и с разной степенью разреженности, оценить время работы всех полученных реализаций и исследовать границы их применимости. Сделать выводы.

In [218]:
def generate_random_graph(n, density):
    matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i, n):
            if np.random.rand() < density:
                matrix[i-1, j-1] = 1
    np.fill_diagonal(matrix, 0)
    matrix += matrix.T
    matrix = Matrix.from_dense(matrix, missing_value=0.0)
    return matrix

In [230]:
generate_random_graph(20, 0.4)

"M_2569270"  nvals  nrows  ncols  dtype   format
gb.Matrix      168     20     20   FP64  bitmapr
------------------------------------------------
     0    1    2    3    4    5    6    7    8    9    10   11   12   13   14  \
0        1.0       1.0  1.0            1.0  1.0                 1.0             
1   1.0            1.0  1.0                 1.0  1.0       1.0  1.0  1.0        
2                  1.0       1.0       1.0                      1.0  1.0  1.0   
3   1.0  1.0  1.0       1.0  1.0                           1.0       1.0        
4   1.0  1.0       1.0       1.0  1.0            1.0  1.0  1.0  1.0             
5             1.0  1.0  1.0       1.0       1.0  1.0            1.0       1.0   
6                       1.0  1.0       1.0  1.0  1.0            1.0  1.0  1.0   
7   1.0       1.0                 1.0       1.0                           1.0   
8   1.0  1.0                 1.0  1.0  1.0       1.0  1.0                       
9        1.0            1.0  1.0  1.0       1.0                                 
10                      1.0                 1.0                      1.0  1.0   
11       1.0       1.0  1.0                                     1.0  1.0        
12  1.0  1.0  1.0       1.0  1.0  1.0                      1.0                  
13       1.0  1.0  1.0            1.0                 1.0  1.0                  
14            1.0            1.0  1.0  1.0            1.0                       
15            1.0       1.0  1.0  1.0            1.0            1.0             
16       1.0                 1.0  1.0  1.0            1.0  1.0       1.0        
17  1.0  1.0            1.0  1.0       1.0       1.0  1.0       1.0  1.0  1.0   
18  1.0  1.0  1.0                                1.0                 1.0        
19                           1.0       1.0                      1.0             

     15   16   17   18   19  
0             1.0  1.0       
1        1.0  1.0  1.0       
2   1.0            1.0       
3                            
4   1.0       1.0            
5   1.0  1.0  1.0       1.0  
6   1.0  1.0                 
7        1.0  1.0       1.0  
8                            
9   1.0       1.0  1.0       
10       1.0  1.0            
11       1.0                 
12  1.0       1.0       1.0  
13       1.0  1.0  1.0       
14            1.0            
15            1.0       1.0  
16                 1.0       
17  1.0                 1.0  
18       1.0            1.0  
19  1.0       1.0  1.0

In [226]:
for i in [100, 200, 500, 1000, 10000]:
  print(f'Количество вершин: {i}, плотность 0.1')
  random_graph = generate_random_graph(i, 0.1)
  print(f'nvals: {(matrix > 0).sum()}, nrows: {matrix.shape[0]}, ncols: {matrix.shape[1]}')
  test(random_graph)
  print()

Количество вершин: 100, плотность 0.1
nvals: 8, nrows: 5, ncols: 5
naiveA3: 202.0; naiveMaskA2: 202.0; cohen: 202.0; sandia: 202.0; naiveA3:
852 µs ± 22.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
naiveMaskA2:
150 µs ± 2.05 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
cohen:
212 µs ± 888 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
sandia:
155 µs ± 433 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

Количество вершин: 200, плотность 0.1
nvals: 8, nrows: 5, ncols: 5
naiveA3: 1339.0; naiveMaskA2: 1339.0; cohen: 1339.0; sandia: 1339.0; naiveA3:
3.14 ms ± 33.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
naiveMaskA2:
481 µs ± 1.63 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
cohen:
375 µs ± 3.07 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
sandia:
228 µs ± 5.19 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

Количество вершин: 500, плотность 0.1
nvals: 8, nrows: 5, ncols: 5
naiveA

С ростом количества вершин, при сохранении степени разреженности, очевидно растет время работы наивного алгоритма в сравнении с другими. Лучшим по понятным причинам становится sandia

In [ ]:
for i in [0.005, 0.05, 0.1, 0.5, 0.8]:
  print(f'Количество вершин: 1000, плотность: {i}')
  random_graph = generate_random_graph(1000, i) 
  print(f'nvals: {(matrix > 0).sum()}, nrows: {matrix.shape[0]}, ncols: {matrix.shape[1]}')
  test(random_graph)
  print()

Количество вершин: 1000, плотность: 0.005
nvals: 8, nrows: 5, ncols: 5
naiveA3: 18.0; naiveMaskA2: 18.0; cohen: 18.0; sandia: 18.0; naiveA3:
1.94 ms ± 44.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
naiveMaskA2:
235 µs ± 2.12 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
cohen:
348 µs ± 9.13 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
sandia:
261 µs ± 17.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

Количество вершин: 1000, плотность: 0.05
nvals: 8, nrows: 5, ncols: 5
naiveA3: 20437.0; naiveMaskA2: 20437.0; cohen: 20437.0; sandia: 20437.0; naiveA3:
106 ms ± 395 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
naiveMaskA2:
2.21 ms ± 124 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
cohen:


С уменьшением разреженности матрицы, при сохранении количества вершин, пропорционально растет время работы всех алгоритмов и мы видимо, что более совершенный алгоритм работает быстрее предыдущего, что логично

На практике, применение наивного алгоритма нелогично ни при каких адекватных исходных данных, поскольку алгоритм с маской будет превосходить, при большом количестве ненулевых данных необходимо использовать sandia, при малой разреженности тоже лучше использовать sandia - другие могут просто не сойтись